선형 회귀
- x: 훈련 세트, 하나의 특성 x1을 갖는 100개의 데이터
- y: 100개의 레이블, 기본적으로 4 + 3 * x의 형식을 따르나, 훈련을 위해 잡음(noise)를 추가한다.

In [ ]:
import numpy as np

X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

In [ ]:
linreg_data = np.c_[X, y]

In [ ]:
linreg_data[:5]

특성 x1과 레이블 y의 관계를 그리면 다음과 같다.
- 기본적으로 y = 4 + 3 * x의 선형관계를 갖지만, 잡음으로 인한 데이터가 퍼져있다.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(X, y, "b.") # 파랑 점 : 훈련 세트 산점도
plt.xlabel("$x_1$", fontsize=18) # x축 표시
plt.ylabel("$y$", rotation=0, fontsize=18) # y축 표시
plt.axis([0, 2, 0, 15]) # x축, y축 구간 지점
plt.show()

정규 방정식

In [ ]:
X_b = np.c_[np.ones((100, 1)), X] # 모든 샘플에 x0 = 1추가
X_b[:5]

In [ ]:
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
# np.linalg.inv => 괄호 안에 주어진 행렬의 역행렬ㅇ르 계산
print(theta_best)

In [ ]:
X_new = np.array([[0], [2]])
X_new_b = np.c_[np.ones((2, 1)), X_new] # 모든 샘플에 x0 = 1추가
print(X_new_b)

In [ ]:
y_predict = X_new_b.dot(theta_best)
print(y_predict)

In [ ]:
plt.plot(X_new, y_predict, "r-", linewidth=2, label="Predictions") # 빨강 직선, label은 범례 지정용
plt.plot(X, y, "b.") # 파란 점: 휸련 세트 산점도
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)

plt.legend(loc="upper left", fontsize=14) # 범례 위치
plt.axis([0 ,2, 0, 15]) # x축, y축 구간 지정

plt.show()

사이킷런의 LinearRegression 모델
- 훈련된 모델의 iontercept_와 coef_ 속성에 절편과 기울기가 저장된다.

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X,y)
lin_reg.intercept_, lin_reg.coef_

x1=0과 x1=2에 대한 예측값과 앞서 수동으로 계산된 결과는 동일하다.

In [ ]:
lin_reg.predict(X_new)

LinearRgression 클래스 scipy.linalg.lstsq() 함수를 사용한다.
- 즉, 위 함수를 직접 사용할 수 있다.
- numpy.linalg.lstsq() 함수도 동일하게 작동한다.

In [ ]:
import scipy
theta_best_svd, residuals, rank, s = scipy.linalg.lstsq(X_b, y)
print(theta_best_svd)

In [ ]:
theta_best_svd, residuals, rank, s = np.linalg.lstsq(X_b, y, rcond=1e-6)
print(theta_best_svd)

무어-펜로즈의 유사 역행렬은 np.linalg.pinv()를 사용하여 구할 수 있다.

In [ ]:
np.linalg.pinv(X_b).dot(y)

경사 하강법
- 무어-펜로즈 유사 역행렬을 구하는 알고리즘의 계산 복잡도는 O(n^2)이다.
    - 즉, 특성의 수가 늘어날수록 계산 시간의 제곱으로 늘어난다.
    - 많은 수의 특성을 사용하는 데이터에 대해선, SVD 방식을 사용해야 하는 사이킷런의 LinearRegression 모델을 사용할 수 없으며, 배치 경사 하강법에 기반한 모델을 대신 사용해야 한다.

배치 경사 하강법
- 임의로 지정된 파라미터로 시작한 후, MSE가 작아지는 방향으로 파라미터를 조금씩 수정한다.

In [ ]:
eta = 0.1 # 학습률
n_iterations = 1000 # 1000번 파라미터 조정
m = 100 # 샘플 수

theta = np.random.randn(2, 1) # 파라미터 무작위 초기화

for iteration in range(n_iterations):
    gradients = 2 / m * X_b.T.dot(X_b.dot(theta) - y) # 비용 함수 그레이디언트
    theta = theta - eta * gradients # 파라미터 업데이트

print(theta) # 최적의 파라미터

학습률과 모델 학습
- 학습률에 따라 파라미터 학습과정이 어떻게 달라질 수 있는지 보여준다.
    - theta_path_bgd: 배치 경사 하강법에 조정되는 파라미터를 순차적으로 저장하는 용도의 리스트이다.
        - 아래에서 세 종류의 경사 하강법을 비교하는 그림에서 활용된다.
    - plot_gradient_descent(): 선형 호귀 모델 훈련 과정의 처음 10단계를 도표로 그려주는 함수이다.

In [ ]:
theta_path_bgd = []

def plot_gradient_descent(theta, eta, theta_path=None):
    m=len(X_b)
    plt.plot(X, y, "b.") # 훈련 세트 산점도

    n_iterations = 1000 # 1000번 반복 훈련
    for iteration in range(n_iterations):

        # 초반 10번 선형 모델(직선) 그리기
        if iteration < 10:
            y_predict = X_new_b.dot(theta)
            style = "b-" if iteration > 0 else "r--"
            plt.plot(X_new, y_predict, style)
        
        # 파라미터 조정
        gradients = 2 / m * X_b.T.dot(X_b.dot(theta) - y)
        theta = theta - eta * gradients

        # 조정되는 파라미터를 모두 리스트에 저장한다.(theta_path=None 옵션이 아닌 경우)
        if theta_path is not None:
            theta_path.append(theta)
    
    plt.xlabel("$x_1$", fontsize=18)
    plt.axis([0, 2, 0, 15])
    plt.title(f"$\eta = {eta}$", fontsize=16)
        

In [ ]:
# 파랑 직선은 초반 10번의 모델 학습 과정을 보여준다. 
# 학습률(eta)에 따라 학습 과정이 다를 수 있음을 보여준다.
np.random.seed(42)
theta = np.random.randn(2, 1) # 무작위 초기화

plt.figure(figsize = (10 , 4)) # 도표 크기 직점

# eta = 0.02
plt.subplot(131); plot_gradient_descent(theta, eta=0.02)
plt.ylabel("$y$", rotation=0, fontsize=18)

# eta = 0.1
plt.subplot(132); plot_gradient_descent(theta, eta=0.1, theta_path=theta_path_bgd)

# eta=0.5
plt.subplot(133); plot_gradient_descent(theta, eta = 0.5)

plt.show()

확률적 경사 하강법
- theta_path_sgd: 확률적 경사 하강법에서 조정되는 파라미터를 순차적으로 저장하는 용도의 리스트이다.
    - 아래에서 세 종류의 경사 하강법을 비교하는 그림에서 활용된다.

In [ ]:
theta_path_sgd = []

m = len(X)
np.random.seed(42)

In [ ]:
n_epochs = 50
t0, t1 = 5, 50 # 학습 스케줄 하이퍼파라미터

def learning_schedule(t):
    return t0 / (t + t1)

In [ ]:
# 모델이 술며값 근처에서 진동하는 것을 확인할 수 있다.
theta = np.random.randn(2, 1)
for epoch in range(n_epochs):
    # 매 샘플에 대하여 그레이디언트 계산 후 파라미터 업데이트
    for i in range(m):
        # 처음 20번 선형 모델 그리기
        if epoch == 0 and i < 20:
            y_predict = X_new_b.dot(theta)
            style="b-" if i > 0 else "r--"
            plt.plot(X_new, y_predict, style)

        # 파라미터 업데이트
            random_index = np.random.randint(m)
            xi = X_b[random_index: random_index + 1]
            yi = y[random_index: random_index+1]

            gradients = 2 * xi.T.dot(xi.dot(theta) - yi) # 하나의 샘플에 대한 그레이디언트 계산
            eta = learning_schedule(epoch * m + 1) # 학습 스케줄을 이용한 학습률 조정
            theta = theta - eta * gradients
            theta_path_sgd.append(theta)

plt.plot(X, y, "b.")
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.axis([0, 2, 0, 15])
plt.show()

print(theta)

사이킷런의 SGDRegressor 모델
- SGDRegressor 모델은 확률적 경사 하강법을 사용하며, 따라서 매우 빠르게 학습한다.

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1, random_state=42)
sgd_reg.fit(X, y.ravel())

In [ ]:
print(sgd_reg.intercept_, sgd_reg.coef_)

미니배치 경사 하강법
- 크기가 20인 배치를 활용하는 미니배치 경사 하강법을 구현한다.
    - theta_path_mgd: 미니배치 경사 하강법에서 조정되는 파라미터를 순차적으로 처리하는 용도의 리스트이다.
        - 아래에서 세 종류의 경사 하강법을 비교하는 그림에서 활용된다.
    - n_iterations = 50: 에포크 수 50
    - minibatch_size = 20: 배치 크기 20

In [ ]:
theta_path_mgd = []

n_iterations = 50
minibatch_size = 20

np.random.seed(42)
theta = np.random.randn(2,1)  # 랜덤 초기화

t0, t1 = 200, 1000
def learning_schedule(t):
    return t0 / (t + t1)

학습 스케줄이 사용된 미니배치 수t에 의존한다.
- t: 훈련에 사용된 미니배치 개수, 학습 스케줄에 사용된다.

In [ ]:
t = 0

for epoch in range(n_iterations):
    
    # 에포크가 바뀔 때마다 훈련 데이터 섞기
    shuffled_indices = np.random.permutation(m)
    X_b_shuffled = X_b[shuffled_indices]
    y_shuffled = y[shuffled_indices]
    
    # 20개 데이터 샘플을 훈련할 때마다 파라미터 업데이트
    for i in range(0, m, minibatch_size):
        t += 1
        xi = X_b_shuffled[i:i+minibatch_size]
        yi = y_shuffled[i:i+minibatch_size]
        gradients = 2/minibatch_size * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(t)        # 학습 스케줄 활용
        theta = theta - eta * gradients
        theta_path_mgd.append(theta)

배치/확률적/미니배치 경사 하강법 파라미터 학습과정 비교
- 아래 코드는 아래 세 개의 변수에 저장된 파라미터 값들을 도표로 보여준다.
    - 파란선: 배치 경사 하강법의 경우이며, 최적의 파라미터에 제대로 수렴한다.
    - 빨강선: 확률적 경사 하강법의 경우이며, 최적의 파라미터 근처에서 심하게 요동친다.
    - 초록선: 미니배치 경사 하강법의 경우이며, 최적의 파라미터 근처에서 상대적으로 덜 요동친다.

In [ ]:
theta_path_bgd = np.array(theta_path_bgd)
theta_path_sgd = np.array(theta_path_sgd)
theta_path_mgd = np.array(theta_path_mgd)

In [ ]:
plt.figure(figsize=(7,4))
plt.plot(theta_path_sgd[:, 0], theta_path_sgd[:, 1], "r-s", linewidth=1, label="Stochastic")
plt.plot(theta_path_mgd[:, 0], theta_path_mgd[:, 1], "g-+", linewidth=2, label="Mini-batch")
plt.plot(theta_path_bgd[:, 0], theta_path_bgd[:, 1], "b-o", linewidth=3, label="Batch")
plt.legend(loc="upper left", fontsize=16)
plt.xlabel(r"$\theta_0$", fontsize=20)
plt.ylabel(r"$\theta_1$   ", fontsize=20, rotation=0)
plt.axis([2.5, 4.5, 2.3, 3.9])

plt.show()

다항 회귀
- 특성이 여러 개일 때 다항 회귀는 이 특성 사이의 관계를 찾을 수 있다.

In [ ]:
np.random.seed(42)
m = 100
X=6*np.random.rand(m, 1) - 3
y=0.5 * X**2 + X + 2 + np.random.randn(m, 1)

In [ ]:
plt.plot(X, y, "b.")
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.axis([-3, 3, 0, 10])
plt.show()

PolynomialFeatures: 다항 회귀 변환기
degree=2: 2차 다항식을 활용하라고 지정하기. 3, 4 등 임의의 양의 정수를 사용할 수 있다.
include_bias = False: 절편(편향)을 변환된 어레이에 추가할지 여부를 지정한다.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X)
print(X[0])
print(X_poly[0]) # X_poly는 이제 특성 X와, X의 제곱을 포함한다.

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)
# intercept_는 절편을, coef_는 독립 변수를 나타낸다.
print(lin_reg.intercept_, lin_reg.coef_)

In [ ]:
X_new = np.linspace(-3, 3, 100).reshape(100, 1) # 새로운 샘플 100개
X_new_poly = poly_features.transform(X_new) # 2차 다항 회귀 모델 데이터로 변환
y_new = lin_reg.predict(X_new_poly) # 예측하기

plt.plot(X, y, "b.")                                           # 기존 데이터 산점도
plt.plot(X_new, y_new, "r-", linewidth=2, label="Predictions") # 예값 그래프

plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.legend(loc="upper left", fontsize=14)
plt.axis([-3, 3, 0, 10])
plt.show()

학습 곡선
- 아래 코드는 아래의 세 모델을 이용하여 얻어진 예측 값을 비교한다.
    - 초록 실선: 300차 다항 회귀 모델
    - 파랑 파선: 2차 다항 회귀
    - 빨강 철망: 1차 선형 회귀 모델

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# 세 개의 다항 회귀 모델 지정: 차례대로 300차 다항 회귀, 2차 다항 회귀, 1차 선형 회귀 모델의 예측값
for style, witdh, degree in (("g-", 1, 300), ("b--", 2, 2), ("r-+", 2, 1)):
    polybig_features = PolynomialFeatures(degree=degree, include_bias=False) # 다항 특성 변환기
    std_scaler = StandardScaler()                                            # 표준화 축적 조정
    lin_reg =   LinearRegression()                                           # 선형 회귀 모델

    polynomial_regression = Pipeline([                      # 파이프라인: 전처리 + 선형 회귀 모델
        ("poly_features", polybig_features),
        ("std_scaler", std_scaler),
        ("lin_reg", lin_reg),
    ])
    polynomial_regression.fit(X, y) # 훈련
    y_newbig = polynomial_regression.predict(X_new) # 예측

    plt.plot(X_new, y_newbig, style, label=str(degree), linewidth=witdh) # 그래프 그리기

plt.plot(X, y, "b.", linewidth=3) # 원 데이터 산점도
plt.legend(loc="upper left")
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.axis([-3, 3, 0, 10])
plt.show()

학습 곡선
- 훈련 셑와 검증 세트에 대한 모델 성능의 변화를 추적하는 곡선이다.
- 모델 성능의 변화는 훈련 세트의 크기를 1부터 시작해서 최대값까지 바꾸면서 훈련된 모델의 성능을 훈련 세트와 검증 세트 각각에 대해 측정하는 방식으로 확인한다.
- 아래 코드는 훈련 세트와 검증 세트에 대한 RMSE의 변화를 추적하는 그래프를 그린다.
    - plot_learning_curves()
        - model: 훈련에 사용되는 모델 지정
        - X: 8대 2 비율로 훈련 세트와 검증 세트로 분리되는 데이터 셋
        - y: 8대 2 비율로 분리되는 레이블 셋
        - 하는 일: 모델의 학습 곡선 그래프 그리기

In [ ]:
from sklearn.metrics import mean_squared_error           # MSE 수동 계산
from sklearn.model_selection import train_test_split     # 무작위 샘플링

def plot_learning_curves(model, X, y):
    # 8:2 로 분류
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=10)
    train_errors, val_errors = [], []                    # MSE 추적 장치

    for m in range(1, len(X_train)):                     # m 개의 훈련 샘플을 대상으로 훈련
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        # MSE 기록
        train_errors.append(mean_squared_error(y_train[:m], y_train_predict))
        val_errors.append(mean_squared_error(y_val, y_val_predict))

    plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
    plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")
    plt.legend(loc="upper right", fontsize=14)  
    plt.xlabel("Training set size", fontsize=14)
    plt.ylabel("RMSE", fontsize=14)      

과소 적합 모델의 학습 곡선 특징
- 2차 다항식으로 생성된 데이터셋에 대한 선형 회귀 모델의 학습 곡선은 다음과 같으며, 전형적인 과소 적합의 양상을 보여준다.
    - 훈련 데이터(빨강)에 대한 성능
        - 훈련 세트가 커지면서 RMSE가 커진다.
        - 훈련 세트가 어느 정도 커지면 더이상 RMSE가 변하지 않는다.
    - 검증 데이터(ㅍ랑)에 대한 성능
        - 검증 세트에 대한 성능이 훈련 세트에 대한 성능과 비슷해진다.

In [ ]:
lin_reg = LinearRegression()
plot_learning_curves(lin_reg, X, y)
plt.axis([0, 80, 0, 3])                        
plt.show()          

과대 적합 모델의 학습 곡선 특징
- 훈련 데이터(빨강)에 대한 성능: 훈련 데이터에 대한 RMSE가 매우 낮다.
- 검증 데이터(파랑)에 대한 성능과 차이가 크게 벌어진다.

In [ ]:
from sklearn.pipeline import Pipeline

polynomial_regression = Pipeline([
    ("poly_features", PolynomialFeatures(degree=10, include_bias=False)),
    ("lin_reg", LinearRegression()),
])

plot_learning_curves(polynomial_regression, X, y)
plt.axis([0, 80, 0, 3])
plt.show()

규제를 사용하는 선형 모델
- 릿지 호귀
- 라쏘 회귀
- 엘라스틱 넷

릿지 회귀
- X: 훈련 세트, 크기는 20
- X_new: 100개의 검즘용 데이터

In [ ]:
np.random.seed(42)

m=20
X=3*np.random.rand(m, 1) 
y=1 + 0.5 * X + np.random.randn(m, 1) / 1.5 # 1차 선형회귀 모델을 따르도록 한다. 단, 잡음이 추가된다.
x_new=np.linspace(0, 3, 100).reshape(100, 1) # 0~3 구간에서 균등하게 100개의 검증 데이터를 선택한다.

Ridge 모델
- 릿지 규제를 사용한다.
- solver=cholesky: 원래는 경사하강법을 계산하는 옵티마이저를 지정한다. 하지만, cholesky를 지정하면 정규 방정식을 이용하여 최적의 파라미터를 계산한다.
- alpha=1: 규제 강도를 나타낸다.

In [ ]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=1, solver="cholesky", random_state=42)
ridge_reg.fit(X, y)
ridge_reg.predict([[1.5]])

solver=sag: 경사하강법을 지원하는 다양한 옵티마이저 중 하나이다. 꼐속해서 보다 개선된 옵티마이저가 개발된다.

In [ ]:
ridge_reg=Ridge(alpha=1, solver="sag", random_state=42)
ridge_reg.fit(X, y)
ridge_reg.predict([[1.5]])

아래 코드는 다양한 alpha 값의 영향력을 그래프로 보여준다.
오른쪽 그래프는 10차 다항 회귀가 적용된 경우 3개를 보여준다.
- alpha값을 크게 할 수 록 예측값의 그래프가 직선에 가까워진다.

In [ ]:
from sklearn.linear_model import Ridge

def plot_model(model_class, polynomial, alphas, **model_kargs):
    for alpha, style in zip(alphas, ("b-", "g--", "r:")):
        model = model_class(alpha, **model_kargs) if alpha > 0 else LinearRegression()
        if polynomial:
            model = Pipeline([
                    ("poly_features", PolynomialFeatures(degree=10, include_bias=False)),
                    ("std_scaler", StandardScaler()),           # 표준화 축척 조정
                    ("regul_reg", model),
                ])
        model.fit(X, y)
        y_new_regul = model.predict(X_new)
        lw = 2 if alpha > 0 else 1
        plt.plot(X_new, y_new_regul, style, linewidth=lw, label=r"$\alpha = {}$".format(alpha))
    plt.plot(X, y, "b.", linewidth=3)
    plt.legend(loc="upper left", fontsize=15)
    plt.xlabel("$x_1$", fontsize=18)
    plt.axis([0, 3, 0, 4])

plt.figure(figsize=(8, 4))
plt.subplot(121)
plot_model(Ridge, polynomial=False, alphas=(0, 10, 100), random_state=42)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.subplot(122)
plot_model(Ridge, polynomial=True, alphas=(0, 10**-5, 1), random_state=42)

plt.show()

SGDRegressor와 릿지 회귀
- penalty 하이퍼파라미터를 이용하면, 릿지 회귀와 동일하게 작동한다.
- penalty="l2"가 기본값이다.
- l2: 아래 l2 노름을 가리킨다.
- max_iter=1000과 tol=1e-3이 기본값으로 사용된다.

In [ ]:
sgd_reg = SGDRegressor(penalty="l2", alpha=1, max_iter=1000, tol=1e-3, random_state=42)
sgd_reg.fit(X, y.ravel())
sgd_reg.predict([[1.5]])

라쏘 회귀
- 라쏘 규제를 지원한다.
- alpha=0.1
- 릿지 회귀에 사용된 alpha 값보다 작은 값을 사용한다.
    - 릿지 회귀는 파라미터를 제곱해서 사용하는 반면, 라쏘 회귀는 파라미터의 절댓값을 사용기에, alpha가 작더라도 구제에 큰 영향을 주기 때문이다.

In [ ]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X, y)
lasso_reg.predict([[1.5]])

아래 코드는 라쏘 규제에 사용되는 다양한 alpha 값의 영향을 비교한다.
- 오른쪽 그래프는 10차 다항회귀에 적용된 라쏘 회귀의 모델의 예측값이다.

In [ ]:
from sklearn.linear_model import Lasso

plt.figure(figsize=(8, 4))
plt.subplot(121)
plot_model(Lasso, polynomial=False, alphas=(0, 0.1, 1), random_state=42)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.subplot(122)
plot_model(Lasso, polynomial=True, alphas=(0, 10**-7, 1), random_state=42)
plt.show()

라쏘 회귀와 릿지 회귀의 비교
- 아래 코드는 라쏘 회귀와 릿지 회귀 모델 학습 과정 동안 경사하강법이 적용될 때 파라미터가 움직이는 특성을 비교해서 보여주기 위한 코드이다.
    - 좌측 상단
        - l1 노름의 그레이디언트 벡터의 움직임.
        - theta2 처럼 파라미터 값이 작을 수록 먼저 0에 수렴하는 것이 보여진다.
        - 훈련 과정에서는 별로 중요하지 않은 속성을 무시하는 효과를 보여준다.
    - 우측 상단
        - l1 노름이 추가된 MSE 함수의 그레이디언트 벡터의 움직임이다.
        - theta2가먼저 줄어드는 것을 잘 보여준다.
        - 전역 최저점 근처에서 진동한다.
            - 절댓값의미분이 0에서 정의되어 있지 않기 때문이다.
    - 좌측 하단
        - l2 노름의 그레이디언트 벡터의 움직임이다.
        - 모든 파라미터가 원점에 직선으로 달려가는 형태로 줄어든다.
    - 우측 하단
        - l2 노름이 추가된 MSE 함수의 그레이디언트 벡터의 움직임이다.
        - 파라미터가 전역 최저점 근처에서 느려진다.
            - 그레이디언트가 작아지기 때문

In [ ]:
# 도표에 사용되는 좌표값 모음
t1a, t1b, t2a, t2b = -1, 3, -1.5, 1.5

t1s = np.linspace(t1a, t1b, 500)
t2s = np.linspace(t2a, t2b, 500)
t1, t2 = np.meshgrid(t1s, t2s)
T = np.c_[t1.ravel(), t2.ravel()]
Xr = np.array([[1, 1], [1, -1], [1, 0.5]])
yr = 2 * Xr[:, :1] + 0.5*Xr[:,1:]

J = (1/len(Xr) * np.sum((T.dot(Xr.T) - yr.T)**2, axis=1)).reshape(t1.shape)

N1 = np.linalg.norm(T, ord=1, axis=1).reshape(t1.shape)
N2 = np.linalg.norm(T, ord=2, axis=1).reshape(t1.shape)

t_min_idx=np.unravel_index(np.argmin(J), J.shape)
t1_min, t2_min = t1[t_min_idx], t2[t_min_idx]

t_init=np.array([[0.25], [-1]])

In [ ]:
# 파라미터(theta) 경로 기억 어레이 생성 함수
def bgd_path(theta, X, y, l1, l2, core = 1, eta = 0.05, n_iterations = 200):
    path = [theta]
    for iteration in range(n_iterations):
        gradients = core * 2/len(X) * X.T.dot(X.dot(theta) - y) + l1 * np.sign(theta) + l2 * theta
        theta = theta - eta * gradients
        path.append(theta)
    return np.array(path)

fig, axes = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(10.1, 8))
for i, N, l1, l2, title in ((0, N1, 2., 0, "Lasso"), (1, N2, 0,  2., "Ridge")):
    JR = J + l1 * N1 + l2 * 0.5 * N2**2
    
    tr_min_idx = np.unravel_index(np.argmin(JR), JR.shape)
    t1r_min, t2r_min = t1[tr_min_idx], t2[tr_min_idx]
    
    # 좌측 도표 등고선 용도
    levelsN=np.linspace(0, np.max(N), 10)
    levelsJR=(np.exp(np.linspace(0, 1, 20)) - 1) * (np.max(JR) - np.min(JR)) + np.min(JR)
    
    # 라쏘 또는 릿지 회귀 경사하강법 파라미터 경로
    path_JR = bgd_path(t_init, Xr, yr, l1, l2)
    
    # l1 노름 또는 l2 노름 경사하강법 경로
    path_N = bgd_path(np.array([[2.0], [0.5]]), Xr, yr, np.sign(l1)/3, np.sign(l2), core=0)
    
    # 좌측 도표(상단: 라쏘, 하단: 릿지)
    ax = axes[i, 0] 
    ax.grid(True)
    ax.axhline(y=0, color='k')                                     # x 축
    ax.axvline(x=0, color='k')                                     # y 축
    ax.contourf(t1, t2, N / 2., levels=levelsN)                    # 등고선
    
    ax.plot(path_N[:, 0], path_N[:, 1], "y--")                     # 노랑 점선
    ax.plot(0, 0, "ys")                                            # 원점
    ax.plot(t1_min, t2_min, "ys")                                  # 좌표 (2, 1)
    ax.set_title(r"$\ell_{}$ penalty".format(i + 1), fontsize=16)
    ax.axis([t1a, t1b, t2a, t2b])
    if i == 1:
        ax.set_xlabel(r"$\theta_1$", fontsize=16)
    ax.set_ylabel(r"$\theta_2$", fontsize=16, rotation=0)

    # 우측 도표(상단: 라쏘, 하단: 릿지)
    ax = axes[i, 1]   
    ax.grid(True)
    ax.axhline(y=0, color='k')                                      # x 축
    ax.axvline(x=0, color='k')                                      # y 축
    ax.contourf(t1, t2, JR, levels=levelsJR, alpha=0.9)             # 등고선
    
    ax.plot(path_JR[:, 0], path_JR[:, 1], "w-o")                    # 흰색 점실선
    ax.plot(path_N[:, 0], path_N[:, 1], "y--")                      # 노랑 점선
    ax.plot(0, 0, "ys")                                             # 원점
    ax.plot(t1_min, t2_min, "ys")                                   # 좌표 (2, 1)
    ax.plot(t1r_min, t2r_min, "rs")                                 # 빨강 점
    ax.set_title(title, fontsize=16)
    ax.axis([t1a, t1b, t2a, t2b])
    if i == 1:
        ax.set_xlabel(r"$\theta_1$", fontsize=16)

plt.show()

엘라스틱 넷
- 릿지 회귀와 라쏘 회귀의 절충 모델
- alpha=0.1
    - 규제 강도
- l1_ratio=0.5
    - 혼합 비율 r을 가리킨다.
    - r이 0이면 릿지 회귀와 같고, r=1이면 라쏘 회귀와 같다.

In [ ]:
from sklearn.linear_model import ElasticNet
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
elastic_net.fit(X, y)
elastic_net.predict([[1.5]])

SGDRegressor와 엘라스틱넷
- SGDRegressor 모델의 penalty 하이퍼파라미터를 이용하면 릿지 회귀와 동일하게 작동한다.
    - penalty="elasticnet"
    - lr_ratio = 혼합 비율

In [ ]:
sgd_reg = SGDRegressor(penalty="elasticnet", alpha=0.1, l1_ratio=0.5, max_iter=1000, tol=1e-3, random_state=42)
sgd_reg.fit(X, y.ravel())
sgd_reg.predict([[1.5]])

조기 종료
- 아래 코드는 조기 종료 설명을 위해 사용될 100개의 데이터를 5:5 비율로 훈련/검증 세트로 분할한다.
    - m=100
        - 전체 데이터셋 크기
    - X
        - -3에서 3사이에서 무작위로 100개 값 선택
    - y
        - 2차 다항식을 따르면서 생성된 타깃 100개, 잡음이 추가됨

In [ ]:
np.random.seed(42)
m=100
X=6*np.random.rand(m, 1)-3
y=2+X+0.5+X**2+np.random.randn(m, 1)

# 5:5로 훈련/검증 세트로 분할
X_train, X_val, y_train, y_val = train_test_split(X[:50], y[:50].ravel(), test_size=0.5, random_state=42)

In [ ]:
# 조기 종료 예제
from copy import deepcopy

poly_scaler = Pipeline([
    ("poly_features", PolynomialFeatures(degree=90, include_bias=False)),
    ("std_scaler", StandardScaler())
])

X_train_poly_scaled = poly_scaler.fit_transform(X_train)
X_val_poly_scaled = poly_scaler.transform(X_val)

sgd_reg = SGDRegressor(max_iter=1, tol=None, warm_start=True, penalty=None, learning_rate="constant", eta0=0.0005, random_state=42)

minimum_val_error=float("inf")
best_epoch=None
best_model=None
for epoch in range(1000):
    sgd_reg.fit(X_train_poly_scaled, y_train) # 중지된 곳에서 다시 시작한다.
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)
    val_error=mean_squared_error(y_val, y_val_predict)
    if val_error < minimum_val_error:
        minimum_val_error = val_error
        best_epoch = epoch
        best_model = deepcopy(sgd_reg)

아래 코드는 에포크를 늘려가며 훈련 세트와 검증 세트의 성능(RMSE)을 측정해서 기록한다.
에포크는 최대 500번이다.
이후, 훈련/검증 세트의 기록을 각각 빨강 점선과 파랑 실선 그래프로 그린다.
- sgd_reg: SGD 선형회귀 모델에 사용한다.
    - penalty=None
        - 규제를 사용하지 않는다.
    - learning_rate = constant
        - 학습률을 일정하게 유지한다. 즉, 학습 스케줄을 사용하지 않는다.
    - warm_start=True
        - 에포크마다 경사 하강법을 적용할 때 선형 회귀 모델을 초기화 하지 않고 이전 에포크에서 학습한 내용을 이어받아서 학습한다.

In [ ]:
sgd_reg = SGDRegressor(max_iter=1, tol=None, warm_start=True,
                       penalty=None, learning_rate="constant", eta0=0.0005, random_state=42)

n_epochs = 500

train_errors, val_errors = [], []                    # 훈련/검증 모델 성능 기록 장치

for epoch in range(n_epochs):
    sgd_reg.fit(X_train_poly_scaled, y_train)        # warm_start=True 이기에 학습결과를 이어감.
    y_train_predict = sgd_reg.predict(X_train_poly_scaled)
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)
    train_errors.append(mean_squared_error(y_train, y_train_predict))  # 훈련/검증모델 성능 기록
    val_errors.append(mean_squared_error(y_val, y_val_predict))

best_epoch = np.argmin(val_errors)                  # 최고 성능의 모델 기억해두기
best_val_rmse = np.sqrt(val_errors[best_epoch])

# 아래부터는 그래프 그리는 코드임.

plt.annotate('Best model',                          # 도표에 문장 넣는 기능. 내용과 위치를 정확히 지정함.
             xy=(best_epoch, best_val_rmse),        # 색상, 폰트 크기 등등을 일일이 지정함.
             xytext=(best_epoch, best_val_rmse + 1),
             ha="center",
             arrowprops=dict(facecolor='black', shrink=0.05),
             fontsize=16,
            )

best_val_rmse -= 0.03  # 검정 실선을 잘 보여주기 위해서임.
plt.plot([0, n_epochs], [best_val_rmse, best_val_rmse], "k:", linewidth=2)  # 수평 검정 점선
plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="Validation set")    # 파랑 실선
plt.plot(np.sqrt(train_errors), "r--", linewidth=2, label="Training set")   # 빵강 파선

# 기타 추가 정보
plt.legend(loc="upper right", fontsize=14)  # 범례
plt.xlabel("Epoch", fontsize=14)            # 축 정보
plt.ylabel("RMSE", fontsize=14)
plt.show()

In [ ]:
# 가장 좋은 성능의 모델이 훈련된 에포크 수와 모델은 아래와 같다.
print(best_epoch, best_model)

In [ ]:
# 시그모이드 함수 그래프
t = np.linspace(-10, 10, 100)
sig=1 / (1+np.exp(-t))
plt.figure(figsize=(9,3))
plt.plot([-10, 10], [0, 0], "k-")
plt.plot([-10, 10], [0.5, 0.5], "k:")
plt.plot([-10, 10], [1, 1], "k:")
plt.plot([0, 0], [-1.1, 1.1], "k-")
plt.plot(t, sig, "b-", linewidth=2, label=r"$\sigma(t) = \frac{1}{1 + e^{-t}}$")
plt.xlabel("t")
plt.legend(loc="upper left", fontsize=20)
plt.axis([-10, 10, -0.1, 1.1])
plt.show()

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
print(list(iris.keys()))
X = iris["data"][:, 3:] # 꽃잎의 너비
y = (iris["target"] == 2).astype(int) # Iris-Virginica면 1, 그렇지 않으면 0

Bunch 자료형은 기본적으로 dict 자료형과 동일하다.
- 단, Bunch 자료형은 인덱싱을 속성처럼 처리할 수 있다.
- data키에 해당하는 값은 (150, 4) 모양의 어레이이다.
    - 150: 150개의 붓꽃 샘플
    - 4: 네 개의 특성, 차례대로 꽃받침 길이, 꽃받침 너비, 꽃잎 길이, 꽃잎 너비

In [ ]:
# 아래 코드는 처음 5개의 샘플 데이터를 보여준다.
iris.data[:5]

target키는 150개 붓꽃에 대한 품종을 1차원 어레이로 담고 있다.
- 0: 세토사(iris_setosa)
- 1: 버시컬러(iris-Versicolor)
- 2: 버지니카(iris-Virginica)

언급된 순서대로, 50개씩 세 품종의 타깃지 저장되어 있으며, 처음 5개의 샘플은 세토사 품종임을 확인할 수 있다.

In [ ]:
iris.target[:5]

In [ ]:
# 50번 인덱스부터는 Versicolor 품종이다.
iris.target[50:55]

In [ ]:
# 100번 인덱스부터는 Virginica 품종이다.
iris.target[100:105]

In [ ]:
# 위의 정보들이 DESCR 속성에 저장되어 있다.
print(iris.DESCR)

이진 분류: 로지스틱 회귀
- 기본적으로 이진 분류 모델이며, OvR(일대다)전략을 이용하여 다중 클래스 분류도 지원한다.

예제 1
아래 코드는 이진 분류 설명을 위해 꽃잎 너비 특성을 이용하여 버지니카 품종 여부를 판정하는데에 사용되는 데이터셋을 지정한다.

In [ ]:
X = iris["data"][:, 3:] # 1개의 특성(꽃잎 너비)만 사용한다.
y = (iris["target"]==2).astype(int) # 버지니카 품종일 때 양성(1)

In [ ]:
# LogisticRegression 모델 훈련
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver="lbfgs", random_state=42)
log_reg.fit(X, y)

1000개의 샘플을 새로 생성하여 훈련된 로지스틱 회귀 모델의 예측 결과를 도표로 확인하면 다음과 같다.
- X_new: 0~3구간을 1000등분한 값 1000개, 즉, 꽃잎의 너비를 0에서 3사이에서 1000개 선택한다.
- y_proba: 각 샘플에 대한 양성 확률
- x축: 꽃잎 너비 크기

- 초록 실선: 버지니카(양성) 확률
- 파랑 점선: 버지니카가 아닌(음성 확률)

In [ ]:
X_new = np.linspace(0, 3, 1000).reshape(1000, 1)
y_proba = log_reg.predict_proba(X_new)

plt.plot(X_new, y_proba[:, 1], "g-", linewidth=2, label="Iris Virginica")
plt.plot(X_new, y_proba[:, 0], "b--", linewidth=2, label="Not Iris Virginica")

아래 코드는 많은 정보를 도표와 함께 전달한다.
- y=1.0을 지나는 수평선
- x=1.66 근처의 결정 경계(decision boundary): 검은 수직 점선
- 초록 삼각형: 버지니카 품종
- 파랑 사각형: 버지니카가 아닌 품종

In [ ]:
X_new = np.linspace(0, 3, 1000).reshape(-1, 1)
y_proba = log_reg.predict_proba(X_new)
decision_boundry=X_new[y_proba[:, 1] >= 0.5][0]

plt.figure(figsize=(8, 3))
plt.plot(X[y==0], y[y==0], "bs")
plt.plot(X[y==1], y[y==1], "g^")
plt.plot([decision_boundry, decision_boundry], [-1,2], "k:", linewidth=2)
plt.plot(X_new, y_proba[:, 1], "g-", linewidth=2, label="Iris Virginica")
plt.plot(X_new, y_proba[:, 0], "b--", linewidth=2, label="Not Iris Virginica")
plt.text(decision_boundry+0.02, 0.15, "Decision boundary", fontsize=14, color="k", ha="center")
plt.arrow(decision_boundry[0], 0.08, -0.3, 0, head_width=0.05, head_length=0.1, fc="b", ec="b")
plt.arrow(decision_boundry[0], 0.92, 0.3, 0, head_width=0.05, head_length=0.1, fc="g", ec="g")
plt.xlabel("Petal width(cm)", fontsize=14)
plt.ylabel("Probability", fontsize=14)
plt.legend(loc="center left", fontsize=14)
plt.axis([0, 3, -0.02, 1.02])
plt.show()

In [ ]:
# 양성 확률이 0.5가 되는 꽃잎 너비의 길이 즉, 결정 경계는 1.66이다.
print(decision_boundry)

In [ ]:
# 실제로 0.66보다 크면 양성, 작으면 음성으로 판정한다.
log_reg.predict([[1.7], [1.5]])

예제 2
- 아래 코드는 이진 분류 설명을 위해 꽃잎 너비와 두 개의 특성을 이용하여 버지니카 품조 여부를 판정하는 모델을 훈련한다.

In [ ]:
X = iris["data"][:, (2, 3)] # 꽃잎 길이와 너비
y = (iris["target"] == 2).astype(int)

log_reg = LogisticRegression(solver="lbfgs", C=10**10, random_state=42)
# solver는 로지스틱 회귀 모델을 최적화하기 위해 사용되는 알고리즘이다.
# C는 정구화의 강도를 제어한다. 정규화는 과적합을 방지하기 위해 모델의 복잡성에 제약을 추가하는 방법이다.
log_reg.fit(X, y)

훈련 결과를 도표로 그리면 다음과 같다.
- 검은 실선: 결정 경계
- 다른 직선들: 버지니카(양성) 추정 확률별 결정 경계를 보여준다.

In [ ]:
x0, x1 = np.meshgrid(
    np.linspace(2.9, 7, 500).reshape(-1, 1), # 2.9와 7 사이의 500개의 요소를 갖는 1차원 배열 생성
    np.linspace(0.8, 2.7, 200).reshape(-1, 1),
)
X_new = np.c_[x0.ravel(), x1.ravel()] # ravel() 메소드는 배열을 1차원으로 변환한다.
# np.c_는 Numpy의 특수한 인덱싱 객체로, 배열을 열 방향으로 결합하는데 사용된다.
# 즉, x0.ravel()의 결과와 x1.ravel()의 결과를 열 방향으로 결합하여 새로운 2차원 배열을 만든다.
y_proba = log_reg.predict_proba(X_new)

plt.figure(figsize=(10, 4))
plt.plot(X[y==0, 0], X[y==0, 1], "bs")
plt.plot(X[y==1, 0], X[y==1, 1], "g^")
zz = y_proba[:, 1].reshape(x0.shape)
contour = plt.contour(x0, x1, zz, cmap=plt.cm.brg)

left_right = np.array([2.9, 7])
boundary = -(log_reg.coef_[0][0] * left_right + log_reg.intercept_[0]) / log_reg.coef_[0][1]

plt.clabel(contour, inline=1, fontsize=12)
plt.plot(left_right, boundary, "k--", linewidth=3)
plt.text(3.5, 1.5, "Not Iris Virginica", fontsize=14, color="b", ha="center")
plt.text(6.5, 2.3, "Iris Virginica", fontsize=14, color="g", ha="center")
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.axis([2.9, 7, 0.8, 2.7])
plt.show()

예제 3
- 꽃잎의 길이와 너비 특성을 이용하여 붓꽃 샘플을 세 개의 품종으로 분류한다.
- LogisticRegression 모델을 이용하여 소프트맥스 회귀를 실행할 수 있다.
    - multi_class="multinomial": 소프트맥스 회귀 지정
    - solver="lbfgs": 경사하강법 알고리즘 지정
    - C=10: 릿지 규제 강도, alpha의 역수에 해당한다. 즉, 작을 수록 강한 규제를 나타낸다.

In [ ]:
X = iris["data"][:, (2, 3)] # 꽃잎 길이, 꽃잎 넓이
y = iris["target"]

softmax_reg = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=10, random_state=42)
softmax_reg.fit(X, y)

훈련 결과를 도표로 그리면 다음과 같다.
- 노랑 원: 세토사
- 파랑 사각형: 버시컬러
- 초록 삼각형: 버지니카
- 결정 경계: 색상으로 구분된다.
- 다른 직선들: 버시컬러 품종 추종 확률별 결정 경계를 보여준다.

In [ ]:
x0, x1 = np.meshgrid(
    np.linspace(0, 8, 500).reshape(-1, 1),
    np.linspace(0, 3.5, 200).reshape(-1, 1),
)
X_new = np.c_[x0.ravel(), x1.ravel()]

y_proba = softmax_reg.predict_proba(X_new)
y_predict = softmax_reg.predict(X_new)

zz1 = y_proba[:, 1].reshape(x0.shape)
zz = y_predict.reshape(x0.shape)

plt.figure(figsize=(10, 4))
plt.plot(X[y==2, 0], X[y==2, 1], "g^", label="Iris Virginica")
plt.plot(X[y==1, 0], X[y==1, 1], "bs", label="Iris Versicolor")
plt.plot(X[y==0, 0], X[y==0, 1], "yo", label="Iris Setosa")

from matplotlib.colors import ListedColormap
custom_cmap = ListedColormap(["#fafab0", "#9898ff", "#a0faa0"])

plt.contourf(x0, x1, zz, cmap=custom_cmap)
contour=plt.contour(x0, x1, zz1, cmap=plt.cm.brg)
plt.clabel(contour, inline=1, fontsize=12)
plt.xlabel("Petal Length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.legend(loc="center left", fontsize=14)
plt.axis([0, 7, 0, 3.5])
plt.show()

In [ ]:
# 꽃잎의 길이 5, 꽃잎의 너비 2인 붓꽃의 품종은 버지니카(2)이다.
softmax_reg.predict([[5, 2]])

품종별 추정 확률은 다음과 같다
- 세토사 추정 확률: 0%
- 버시컬러 추정 확률: 5.75%
- 버지니카 추정 확률: 94.25%

In [ ]:
softmax_reg.predict_proba([[5,2]])